# Binance Finance Data

In [ ]:
from binance import Client
import pandas as pd
import numpy as np
import talib as tb
import time
import pandas_ta as pta # to create supertrend indicator   
import plotly.graph_objs as go 

In [ ]:
# API key and secret for the testnet

key = ''
secret = ''

In [ ]:
# Sending request to Binance Testnet to connect to Python

client = Client(api_key=key, api_secret=secret, testnet=True)

In [ ]:
# Getting live data in a data frame format from Binance API and the date into an organized
# and readable DataFrame.

def getdata(symbol, interval, past):
    frame = pd.DataFrame(client.get_historical_klines(symbol, interval, past + ' min ago UTC'))
    frame = frame.iloc[:, :5]
    frame.columns = ['Time', 'Open', 'High', 'Low', 'Close']
    frame = frame.set_index('Time')
    frame.index = pd.to_datetime(frame.index, unit='ms')
    frame = frame.astype(float)
    return frame

# Retrieving and reading the data into a DataFrame for the given symbol, interval, and 
# past time period.

df = getdata('BTCUSDT', '1m', '60000')

df

In [ ]:
# Creating RSI column

def algocalculation(df):
    df['RSI'] = tb.RSI(np.array(df['Close']), timeperiod=5)

In [ ]:
algocalculation(df)

df

In [ ]:
df['Signal'] = 'HOLD'  # Default to HOLD
df.loc[df['RSI'] < 30, 'Signal'] = 'BUY'
df.loc[df['RSI'] > 70, 'Signal'] = 'SELL'
df

## Data Type

In [ ]:
df.info()

In [ ]:
df['Open'] = df['Open'].astype(float)

## NaN Values

In [ ]:
missing_values = df.isnull().sum() 
print(missing_values)

In [ ]:
df.dropna(inplace = True)

## Duplicates

In [ ]:
df.drop_duplicates(inplace=True)

## Dummy Variables

In [ ]:
def delete_flagged_rows(df):
    columns_to_check = ['Open', 'Close', 'High', 'Low']
    for col in columns_to_check:
        mean = df[col].mean()
        std = df[col].std()
        df = df[df[col].between(mean - 3 * std, mean + 3 * std)]
    return df

df = delete_flagged_rows(df)

## Outliers

In [ ]:
Q1 = df['Close'].quantile(0.25) 
Q3 = df['Close'].quantile(0.75) 
IQR = Q3 - Q1 
df = df[~((df['Close'] < (Q1 - 1.5 * IQR)) | (df['Close'] > (Q3 + 1.5 * IQR)))]

## Filtering Data

In [ ]:
df = df.filter(['Time', 'Close', 'Signal'])
df